# Cambiamenti climatici

## Requisiti:

In [12]:
!pip install pandas
!pip install tabulate


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Data collection
I dati usati in questa applicazione si trovano nella cartella [data](applicazione/data/). 
<ul>
    <li>In original si trovano i file originali trovati su internet</li>
    <li>In modified si trovano i file "processati"</li>
</ul>

## Dati sulle temperature e sulle stazioni geografiche
I file contenenti i dati relativi alle temperature ([temperatures.dat](applicazione/data/original/temperatures.dat)) e alle stazioni geografiche ([stations.inv](applicazione/data/original/stations.inv)) sono stati trovati sul sito [National Centers for Environmental Information](https://www.ncei.noaa.gov/). Sono scaricabili al seguente link: https://www.ncei.noaa.gov/pub/data/ghcn/v4/ghcnm.tavg.latest.qcf.tar.gz <br>


## Dati sui Paesi
I file contenenti informazioni sui Paesi sono stati trovati su due repository di github: <br>
- [fips_to_iso_country_codes.csv](applicazione/data/original/fips_to_iso_country_codes.csv) trovato al seguente link: https://github.com/mysociety/gaze/blob/master/data/fips-10-4-to-iso-country-codes.csv <br>
- [countries_info.csv](applicazione/data/original/countries_info.csv) trovato al seguente link https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv <br>

## 2. Data processing

In [13]:
import pandas as pd
import os

DATA_FOLDER = "applicazione" + os.path.sep + "data" + os.path.sep
DATA_ORIGINAL_FOLDER = DATA_FOLDER + "original"+ os.path.sep
DATA_MODIFIED_FOLDER = DATA_FOLDER + "modified"+ os.path.sep

### Stazioni e temperature
I file [stations.inv](applicazione/data/original/stations.inv) e [temperatures.dat](applicazione/data/original/temperatures.dat) trovati su internet sono Fixed Width Text File. Ho deciso di modificarli e convertirli in file CSV ([stations_created.csv](applicazione/data/modified/stations_created.csv), [tempetatures_created.csv](applicazione/data/modified/tempetatures_created.csv)). In questo modo creo due file CSV con i dati già "puliti" e pronti per essere letti. Inoltre, la funzione read_csv per leggere file CSV è molto più veloce di read_fwf per leggere file Fixed Width, quindi <mark> continua... </mark>
<br>

#### Stazioni
Il file [stations_created.csv](applicazione/data/modified/stations_created.csv) ha la seguente struttura:
|Campo| Significato|
|:-|:-|
|station_id| Codice identificativo della stazione. I primi due caratteri sono il codice FIPS del Paese|
|latitude| Latitudine della stazione in gradi decimali|
|longitude| Longitudine della stazione in gradi decimali|
|station_elev| Altitudine della stazione in metri; <mark> FORSE NON E' COSI' il valore -999.0 indica un dato mancante|
|station_name| Nome della stazione|
|fips| Codice FIPS del Paese|

Ho creato uno script per creare il file:

In [48]:
import calendar

# File con le stazioni
FILE_ORIGINALE_STATIONS = DATA_ORIGINAL_FOLDER + "stations.inv"
FILE_CSV_STATIONS = DATA_MODIFIED_FOLDER + "stations_created.csv"
HEADERS_STATIONS = ["station_id","latitude","longitude","station_elev","station_name"]

if not os.path.exists(FILE_CSV_STATIONS):
    print("Creo file csv partendo da",FILE_ORIGINALE_STATIONS,"...")
    
    # Leggo il file
    df = pd.read_fwf(FILE_ORIGINALE_STATIONS,widths=[11,9,10,7,31], header=None, names=HEADERS_STATIONS)
    
    # Aggiungo il campo fips
    df['fips'] = df["station_id"].apply(lambda x : x[0:2])
    HEADERS_STATIONS += ["fips"]
    
    # Trasformo i valori -999.0 di station_elev in Nan
    df["station_elev"] = df["station_elev"].apply(lambda x: pd.NA if x == 9999 else x)

    # Converto in formato CSV
    df.to_csv(FILE_CSV_STATIONS, index=False)
print("File creato correttamente!")

Creo file csv partendo da applicazione\data\original\stations.inv ...
File creato correttamente!


#### tempetatures_created.csv
Il file [tempetatures_created.csv](applicazione/data/modified/stations_created.csv) ha la seguente struttura:
|Campo| Significato|
|:-|:-|
|station_id| Codice identificativo della stazione. I primi due caratteri sono il codice FIPS del Paese|
|year| Anno in cui sono state misurate le temperature|
|month_value| Valore della temperatura del mese "month". I valori della temperatura sono in centesimi di grado Celsius, ma sono espressi come numeri interi (ad es. dividere per 100,0 per ottenere gradi Celsius interi). <br>Il valore -9999 indica un dato mancante. |
|<mark>TOLTO month_dm_flag| data measurement flag (indicatore della misurazione dei dati). <br>Se è vuoto -> nessuna informazione di misurazione applicabile <br> Se è una lettera tra a-i -> numero di giorni mancanti nel calcolo della temperatura media mensile |
|<mark>CAMBIATO month_qc_flag| quality control flag del mese "month". <br> Se è vuoto, non c'è stato nessun fallimento del controllo di qualità oppure il valore non può essere valutato. <br> Per gli altri valori, consultare la [documentazione dei dataset]((https://www.ncei.noaa.gov/pub/data/ghcn/v4/readme.txt)) |
|<mark>TOLTO month_ds_flag| Indica l'origine dati per il valore del mese "month". Per avere altre informazioni, consultare la [documentazione di tale campo](https://www.ncei.noaa.gov/pub/data/ghcn/v4/ghcnm-flags.txt)|

Ho creato uno script per creare il file:


In [50]:
# File con le temperature
FILE_ORIGINALE_TEMPERATURES = DATA_ORIGINAL_FOLDER + "temperatures.dat"
FILE_CSV_TEMPERATURES = DATA_MODIFIED_FOLDER + "temperatures_created.csv"

# Creo due liste: una con le lunghezze dei campi e una con i nomi delle colonne
list_width= [11,4,4] + [j for i in [[5,1,1,1] for _ in range(12)] for j in i] 
len(list_width)
HEADERS_TEMPERATURES = ["station_id", "year", "DEL_ME"]
for i in range(1,13):
    month_name = calendar.month_name[i]
    HEADERS_TEMPERATURES += [month_name+"_value","DEL_ME"+str(i)+"a",month_name+"_qc_flag","DEL_ME"+str(i)+"b"]

if not os.path.exists(FILE_CSV_TEMPERATURES):
    print("Creo file csv partendo da",FILE_ORIGINALE_TEMPERATURES,"...")
    
    # Leggo il file
    df = pd.read_fwf(FILE_ORIGINALE_TEMPERATURES, widths=list_width,header=None, names=HEADERS_TEMPERATURES)
    
    # Elimino le colonne non necessarie
    HEADERS_TEMPERATURES = [i for i in HEADERS_TEMPERATURES if not i.__contains__("DEL_ME")]
    df = df[HEADERS_TEMPERATURES]

    # Cambio i valori valori "vuoti" di month_qc_flag con un trattino. Questi valori vuoti, indicano che 
    # non ci sono stati problemi durante la rilevazione dei dati, non indicano dati mancanti.
    # Cambio i valori -9999 di month_value con NaN. Trasformo anche i valori validi in gradi decimali. 
    for i in range(1,13):
        month_name = calendar.month_name[i]
        df[month_name+"_qc_flag"] = df[month_name+"_qc_flag"].apply(lambda x: "-" if pd.isna(x) else x)
        df[month_name+"_value"] = df[month_name+"_value"].apply(lambda x: pd.NA if str(x) == "-9999" else x/100)

    # Converto in formato CSV
    df.to_csv(FILE_CSV_TEMPERATURES, index=False)

print("File creato correttamente!")

Creo file csv partendo da applicazione\data\original\temperatures.dat ...
File creato correttamente!


### Paesi
Per quanto riguarda le informazioni sui paesi, ho unito i due file [country_info.csv](applicazione/data/original/countries_info.csv) e [fips_to_iso_country_codes.csv](applicazione/data/original/fips_to_iso_country_codes.csv) in un unico file [countries_created.csv](applicazione/data/original/countries_created.csv) con la seguente struttura:

|Campo|Significato|
|:-|:-|
|fips| Codice FIPS del paese|
|name_country| Nome del Paese|
|region| Continente di appartenenza|
|sub-region| Zona del continente (nord, sud, est, ovest)|
|region-code| Codice di region|
|sub-region-code| Codice di sub-region|

Ho creato uno script per creare il file:

In [ ]:
# File dei paesi
FILE_ORIGINALE_FIPS_TO_ISO = DATA_ORIGINAL_FOLDER + "fips_to_iso_country_codes.csv"
FILE_ORIGINALE_COUNTRY_INFO = DATA_ORIGINAL_FOLDER + "countries_info.csv"
FILE_CSV_COUNTRIES_CREATED = DATA_MODIFIED_FOLDER + "countries_created.csv"

if not os.path.exists(FILE_CSV_COUNTRIES_CREATED):
    print("Creo file csv partendo da",FILE_ORIGINALE_FIPS_TO_ISO, "e", FILE_ORIGINALE_COUNTRY_INFO,"...")

    # Leggo i files originali
    df_fips = pd.read_csv(FILE_ORIGINALE_FIPS_TO_ISO)
    df_country = pd.read_csv(FILE_ORIGINALE_COUNTRY_INFO)

    # Faccio il join sul codice alpha-2
    merged = pd.merge(df_fips, df_country, how='left',left_on='ISO 3166', right_on='alpha-2')

    # Elimino le colonne non necessarie e rinomino le altre
    merged = merged.drop(['ISO 3166','iso_3166-2', 'alpha-3', 'alpha-2', 'country-code', 'intermediate-region', 'intermediate-region-code'], axis=1)
    merged['name_country'] = merged.apply(lambda x : str(x['Name'] if pd.isna(x["name"]) else x['name']), axis=1)
    merged = merged.drop([ 'name', 'Name'], axis=1)
    merged = merged.rename(columns={"FIPS 10-4":"fips"})

    # e creo un nuovo file
    merged.to_csv(FILE_CSV_COUNTRIES_CREATED, index=False, header=True)

print("File creato correttamente!")

File creato correttamente!


## 3. Elaborazione dei dati <mark>(?)

Una volta creati tutti i file CSV, per usarli basterà leggerli e fare il join sul campo 'fips' e 'station_id':

In [ ]:
df_temperatues = pd.read_csv(FILE_CSV_TEMPERATURES)
df_stations = pd.read_csv(FILE_CSV_STATIONS)
df_country = pd.read_csv(FILE_CSV_COUNTRIES_CREATED)

In [35]:
import tabulate

df_merged = df_temperatues.merge(df_stations, on='station_id')
df_merged = df_merged.merge(df_country, on='fips')
colonne = ["station_id", "year", "station_name", "latitude", "longitude", "station_elev", "region"] + [str(calendar.month_name[i]+"_value") for i in range(1,13)]
df_merged = df_merged[colonne]


table = tabulate.tabulate(df_merged.sort_values(by="March_value").head(5), headers='keys', tablefmt='html')
display(table)

,station_id,year,station_name,latitude,longitude,station_elev,region,January_value,February_value,March_value,April_value,May_value,June_value,July_value,August_value,September_value,October_value,November_value,December_value
92097,AYM00089606,1960,VOSTOK,-78.45,106.867,3488,nan,-3609,-4445,-6210,-6794,-6800,-7075,-6814,-7135,-6630,-5905,-4600,-3434
92110,AYM00089606,1974,VOSTOK,-78.45,106.867,3488,nan,-2959,-4705,-6169,-6860,-6725,-6244,-6590,-6830,-6815,-5495,-4250,-3255
92000,AYM00089577,2013,DOME_PLATEAU_DOME_A,-80.37,77.37,4084,nan,-3420,-4743,-6168,-6489,-6665,-6029,-6708,-6222,-5819,-5409,-3972,-3083
92148,AYM00089606,2013,VOSTOK,-78.45,106.867,3488,nan,-3230,-4700,-6110,-6401,-6660,-6112,-6555,-6290,-5924,-5540,-3980,-3060
92001,AYM00089577,2014,DOME_PLATEAU_DOME_A,-80.37,77.37,4084,nan,-3209,-4521,-6074,-6474,-6151,-6330,-6783,-6408,-5974,-5555,-4105,-3291


## 

In [ ]:
df_temperatues = pd.read_csv(FILE_CSV_TEMPERATURES)

KeyboardInterrupt: 

In [ ]:
for i in range(1,13):
    month_name = calendar.month_name[i]
    print("********")
    new_df  = df_temperatues
    
    
    new_df[month_name+"_qc_flag"] = df_temperatues[month_name+"_qc_flag"].apply(lambda x: "-" if pd.isna(x) else x)
    prova = new_df[new_df[month_name+"_qc_flag"] == "-"]
   # lista = list(set(prova.to_list()))
    print(month_name, len(prova))

    

********
January 1345254
********
February 1343882
********
March 1344461
********
April 1343911
********


KeyboardInterrupt: 

In [ ]:

# for index, row in merged.iterrows():
#     print(f"{index:5}", f"{row['name']:50}",row['fips'])


# print(merged)

df_stations = pd.read_csv(FILE_CSV_STATIONS)
# print(len(pd.unique(df_stations['fips'])))

merged2 = df_stations.merge(merged, how='left', on='fips')
# merged2 = merged2.drop(['name_country', 'region', 'sub-region', 'region-code', 'sub-region-code'], axis=1)
merged2 = merged2.drop_duplicates() 
merged2 = merged2.reset_index()

# print(merged2)
for index, row in merged2.iterrows():
    print(str(row).replace("\n","\t",-1))

